You may find this series of notebooks at [databricks-solutions/realtime-rag-agents-databricks-youcom](https://github.com/databricks-solutions/realtime-rag-agents-databricks-youcom). For more information about this solution accelerator, visit the [blog post](https://you.com/articles/unlocking-real-time-intelligence-for-ai-agents-with-you.com-and-databricks).

# Evaluate Agent

Now that we have our agent define, let's evaluate it's performance based on manual traces and using [FreshQA's](https://github.com/freshllms/freshqa?tab=readme-ov-file) evaluation dataset.


## 1. Create an Evaluation Dataset from Traces

MLflow Evaluation Datasets can be created via multiple approaches. The example below creates an empty Evaluation dataset then populuates from the MLflow Traces captured in the above predictions.

In [0]:
if not spark.catalog.tableExists(evaluation_dataset_path):
    assert run_id is not None, "Run ID has not been specified. Please run the cell above"

    eval_dataset = mlflow.genai.datasets.create_dataset(
        uc_table_name=evaluation_dataset_path,
    )

    traces = mlflow.search_traces(run_id=run_id)

    eval_dataset.merge_records(traces)

eval_dataset = mlflow.genai.datasets.get_dataset(uc_table_name=evaluation_dataset_path)
display(eval_dataset.to_df())

## 2. Conduct Evaluation

Evaluate using the evaluation dataset established in `1. Create an Evaluation Dataset from Traces`

In [0]:
scorers = [
        RetrievalGroundedness(),  # Checks if email content is grounded in retrieved data
        Guidelines(
            name="professional_tone",
            guidelines="The generated response must be in a professional tone.",
        ),
        RelevanceToQuery(),  # Checks if email addresses the user's request
        Safety(),  # Checks for harmful or inappropriate content
    ]

# Run evaluation with predefined scorers
eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda messages: AGENT.predict({"messages": messages}),
    scorers=scorers,
)

## 3. FreshQA Evaluation

Load a recent [FreshQA dataset](https://github.com/freshllms/freshqa?tab=readme-ov-file). Create a dataset of the questions and expected answers. Perform an Evaluation using the AGENT.

In [0]:

def get_or_create_freshqa_eval_dataset(spark,
                                       table_path,
                                       doc_id="1XYTgnilxwoxihTBYg9X3II4a4orHQYLAkyVnf70LA1c", 
                                       sheet_id="334049794"):

  if not spark.catalog.tableExists(table_path):
  
    # https://github.com/freshllms/freshqa?tab=readme-ov-file
    # FreshQA July 28, 2025 - https://docs.google.com/spreadsheets/d/1XYTgnilxwoxihTBYg9X3II4a4orHQYLAkyVnf70LA1c/edit?gid=334049794#gid=334049794

    freshqa_dataset_path = f"https://docs.google.com/spreadsheets/d/{doc_id}/export?format=csv&gid={sheet_id}"
    response = requests.get(freshqa_dataset_path)
    assert response.status_code == 200, f"Encountered error when fetching FreshQA dataset. Please check that '{freshqa_dataset_path}' exists."

    with io.BytesIO(response.content) as csv_file:
      pdf = pd.read_csv(csv_file, header=2)

    # select the TEST split dataset
    test_pdf = pdf[pdf["split"] == "TEST"].copy().reset_index(drop=True)

    # convert the FreshQA dataset into the format expected by mlflow.genai.evaluate
    eval_dataset_pdf = pd.DataFrame()
    eval_dataset_pdf["inputs"] = test_pdf["question"].apply(lambda x: create_message_payload(x))
    eval_dataset_pdf["expectations"] = test_pdf["answer_0"].apply(lambda x: {"expected_response": x})

    eval_dataset = mlflow.genai.datasets.create_dataset(uc_table_name=table_path)

    eval_dataset.merge_records(eval_dataset_pdf)

  return mlflow.genai.datasets.get_dataset(table_path)

In [0]:
freshqa_dataset_table_name = "freshqa_eval_set"
freshqa_eval_dataset_path = f"{catalog}.{schema}.{freshqa_dataset_table_name}"

# freshQA Google Sheet as of July 28, 2025
# This can be updated by accessing the Google Sheet from this link
# https://github.com/freshllms/freshqa?tab=readme-ov-file
freshqa_doc_id = "10c1ZhL091BQmLTQq8ryC_JQex_hKNa0r_lk2-JEDWHM"
freshqa_sheet_id = "334049794"

freshqa_eval_dataset = get_or_create_freshqa_eval_dataset(spark, freshqa_eval_dataset_path, doc_id=freshqa_doc_id, sheet_id=freshqa_sheet_id)

display(eval_dataset.to_df())

In [0]:
scorers = [
        RetrievalGroundedness(),  # Checks if email content is grounded in retrieved data
        Guidelines(
            name="professional_tone",
            guidelines="The generated response must be in a professional tone.",
        ),
        RelevanceToQuery(),  # Checks if email addresses the user's request
        Safety(),  # Checks for harmful or inappropriate content
        Correctness()
    ]

# Run evaluation with predefined scorers
eval_results = mlflow.genai.evaluate(
    data=freshqa_eval_dataset,
    predict_fn=lambda messages: AGENT.predict({"messages": messages}),
    scorers=scorers,
)